# **Image Identifier Models**

This notebook is designed to evaluate and identify characters in movie frames using Generative and discrimiantive models. The notebook is structured to align with the objectives outlined in the thesis.



## **1. Environment Setup**
Install necessary libraries and clone the required repository.

In [ ]:
!git clone https://github.com/Reouth/Movie-Character-Identification-With-Perosnalized-Generative-Models.git

%pip install -qq git+https://github.com/huggingface/diffusers.git
%pip install -q accelerate
!pip install bitsandbytes
!pip install git+https://github.com/openai/CLIP.git


## **2. Import Libraries**
Load necessary Python libraries and scripts.

In [ ]:
import os
import torch
import gc
import configparser
from torch import autocast
import numpy as np
import pandas as pd

# Change directory to cloned repository
os.chdir('/content/Movie-Character-Identification-With-Perosnalized-Generative-Models')

from models import clip_model
from models import Diffusion_Identifier
from src import data_upload
from src import helper_functions


## **3. Configure Authentication**
Login to Hugging Face to access the Stable Diffusion model.

In [10]:
from huggingface_hub import notebook_login
!git config --global credential.helper store
notebook_login()

## **4. Mount Google Drive**
Store and retrieve files from Google Drive.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive



## **5. General Configuration**



**Input/output Path configurations and load**

Note:
* Input path for Diffusion model is ID image-text embedding folder
* Input path for CLIP model is folder for ID images

In [14]:


output_dir = '/content/drive/MyDrive/thesis_OO_SD/testing/Fabelmans_movie/csv_results'#@param {type:"string"} #output main folder
os.makedirs(output_dir,exist_ok=True)
input_dir =  '/content/drive/MyDrive/thesis_OO_SD/Fabelmans_movie/Imagic_embeddings/4_ID/1024res/embeds_with_pipe' #@param {type:"string"} #input main folder

text_prompts  =  ["a photo of a person"] #@param {type:"raw"}  # add text prompts to generate here (correlates to embedings text)


# input/output path creation
input_dirs=[]
output_dirs=[]

for text in text_prompts:
  input_dirs.append(os.path.join(input_dir,text.replace(" ","_")))
  output_dirs.append(os.path.join(output_dir,text.replace(" ","_")))

## **6. Load image test set**


**Image Classes Configuartion and load**

Input images (movie frames) are loaded from a directory in drive and saved as new name "{class name (folder)}_{number of frame}:

*   image_list = [(new_name,PIL image)...]


In [15]:

class_batch = 5 #@param {type:"number"}  #number of images from each class to process  if all set to float('inf')
max_frames = float('inf') #@param {type:"number"} #maximum number of images to process if all set to float('inf')
image_data_path = '/content/drive/MyDrive/thesis_OO_SD/Fabelmans_movie/frames_classes_GT' #@param {type:"string"} #folder to dataset image classes


In [ ]:
image_list =data_upload.upload_images(image_data_path,class_batch =class_batch,max_frames=max_frames)

## **7. Image Identificaton**

### **Diffusion Model Identification for multiple images**

**Diffusion Identifier model configurations**


In [18]:
num_inference_steps = 50  #@param {type:"number"}
resolution = 512  #@param {type:"number"}
height = 512  #@param {type:"number"}
width = 512  #@param {type:"number"}
alpha = 0 #@param {type:"number"}
guidance_scale = 7.5 #@param {type:"number"}
seed = 9 #@param {type:"number"}

cat_embeds = False  #@param {type:"boolean"}
Imagic_pipe = True  #@param {type:"boolean"}
SD_model_name = 'CompVis/stable-diffusion-v1-4' #@param {type:"string"}
CLIP_model_name = 'openai/clip-vit-large-patch14' #@param {type:"string"}
device = "cuda" if torch.cuda.is_available() else "cpu"


**Diffusion Identifier model inference**

* Loss is saved to CSV per class in the output directory


In [ ]:
for input_dir, output_dir in zip(input_dirs,output_dirs):
  os.makedirs(output_dir,exist_ok=True)
  sd_identifier = Diffusion_Identifier.multi_image_identifier(input_dir,output_dir, SD_model_name,CLIP_model_name,
                                                            device,image_list,cat_embeds,Imagic_pipe,alpha,seed=seed,
                                                            height=height,width=width, resolution=resolution,
                                                            num_inference_steps=num_inference_steps,guidance_scale=guidance_scale
                                                            )


### **CLIP Model Identification for multiple image**

**CLIP Identifier model configurations**


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = 'ViT-B/32'  #@param {type:"string"}


**CLIP Identifier model inference**

* Loss is saved to CSV per class in the output directory


In [ ]:
for image_ID_path, csv_folder in zip(input_dirs,output_dirs):
  os.makedirs(csv_folder, exist_ok=True)
  clip_pipeline = clip_model.CLIPPipeline(device,model_name)
  for image_name,image,_ in image_list:
    cls = image_name.rsplit("_",1)[0]
    image_flag, df_clip,csv_file_path = helper_functions.csv_checkpoint(csv_folder,cls,image_name)
    if image_flag:
      continue
    else:
      clip_embeddings = clip_pipeline.images_to_embeddings(image_ID_path)
      clip_classsifed = clip_pipeline.conditioned_classifier(image,clip_embeddings)

      df_image = pd.DataFrame(clip_classsifed,columns = ['input_CLIP_embeds','CLIP_loss'])
      df_image.insert(0, 'GT Image name', image_name)
      df_clip = pd.concat([df_clip, df_image], ignore_index=False)

      # Save to CSV
      df_clip.to_csv(csv_file_path)
